<a href="https://colab.research.google.com/github/JetsadaWijit/university-bangkok-2024-3yr-sm2-cs462-project/blob/master/SuperStoreSalesDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset: [SuperStoreSalesDataset](https://www.kaggle.com/datasets/aditisaxena20/superstore-sales-dataset)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
# Import the kaggle library
%pip install kaggle

if not os.path.exists("data/"):
  # Download the dataset
  !kaggle datasets download -d aditisaxena20/superstore-sales-dataset -p data/

  # Extract the downloaded ZIP file (if necessary)
  !unzip data/superstore-sales-dataset.zip -d data/
else:
  print("Data already downloaded. Skipping download step.")

In [ ]:
df = pd.read_csv("data/SuperStore_Orders.csv", encoding='latin-1')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df.size

In [ ]:
df.info()

**Segment 1 Filtered Data**

In [ ]:
# Drop unwanted columns (assuming they exist)
columns_to_drop = ['ship_date', 'ship_mode', 'customer_name', 'segment', 'state',
                   'product_name', 'shipping_cost', 'order_priority']
filtered_data = df.drop(columns=columns_to_drop)

# Handle potential errors during conversion and filtering
try:
  # Attempt conversion to datetime format, handling various formats
  filtered_data['order_date'] = pd.to_datetime(filtered_data['order_date'], errors='coerce')

  # Filter based on a valid date format (YYYY-MM-DD) after successful conversion
  filtered_data = filtered_data[filtered_data['order_date'].notna() &  # Filter out NaNs
                                (filtered_data['order_date'] >= pd.to_datetime('2014-09-01'))]
except:
  print("Error: Could not convert 'order_date' column to datetime format. Check data format.")

# Drop rows with missing values (after potential filtering)
filtered_data.dropna()

In [ ]:
filtered_data.head()

In [ ]:
filtered_data.tail()

**Segment 2 Build data**

In [ ]:
# Assuming 'sales' is the column with comma-separated values
def extract_sales(sales_string):
  try:
    # Split the string by comma and convert each element to a float
    sales_list = [float(value.strip()) for value in sales_string.split(',')]
    # Calculate the mean of the list
    return sum(sales_list) / len(sales_list)
  except:
    # Handle potential errors during conversion (optional)
    return None  # Or handle differently (e.g., set to 0)

# Apply the function to each row in the 'sales' column
filtered_data['sales'] = filtered_data['sales'].apply(extract_sales)

In [ ]:
mean_sales = filtered_data.groupby(['country', 'category', 'sub_category'])['sales'].mean().reset_index()

In [ ]:
mean_sales

In [ ]:
mean_sales[mean_sales['sales'] == mean_sales['sales'].max()]

In [ ]:
mean_sales[mean_sales['sales'] == mean_sales['sales'].min()]

In [ ]:
# Assuming 'country' is the first column and 'sales' is the column with mean sales
plt.figure(figsize=(12, 6))  # Adjust figure size as needed
plt.bar(mean_sales['country'], mean_sales['sales'])
plt.xlabel('Country')
plt.ylabel('Mean Sales')
plt.title('Mean Sales by Country')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

In [ ]:
# check NaN of null data
filtered_data.isna().sum()

In [ ]:
# check number of Null data
filtered_data.isnull().sum()

**Segment 3 unsupervice**

In [ ]:
# Create a KMeans object with the desired number of clusters
kmeans = KMeans(n_clusters=3)

# Fit the KMeans model to the DataFrame
kmeans.fit(filtered_data[['sales']])

# Add a new column to the filtered_data DataFrame with cluster labels
filtered_data['cluster'] = kmeans.labels_

# Merge the cluster labels back to the original DataFrame df using the index
df = df.merge(filtered_data[['cluster']], how='left', left_index=True, right_index=True)

# Replace NaN values in cluster_x and cluster_y with a placeholder value
df['cluster_x'].fillna(-1, inplace=True)
df['cluster_y'].fillna(-1, inplace=True)

# Print the DataFrame with cluster labels
df
